In [11]:
import os
import numpy as np
from torchsummary import summary

import matplotlib.pyplot as plt
import cv2
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout2d
from torch.optim import Adam, SGD
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from skimage.transform import rotate, AffineTransform, warp
import skimage.io as io
import ctypes
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.transforms as transforms
from scipy.ndimage import rotate
import torch.nn.functional as F
from random import random
np.set_printoptions(threshold=np.inf)
labels = np.load('brain_tumor_dataset/labels.npy')
images = np.load('brain_tumor_dataset/images.npy')
masks = np.load('brain_tumor_dataset/masks.npy')

In [12]:
# vector X and Y contains all the training images and labels of the augmentatied dataset
img = []
lab = []
for itr in tqdm(range(len(labels))):
    temp = []
    temp.append(images[itr,:,:])
    img.append(temp)
    lab.append(labels[itr]-1)
    
    temp=[]
    temp.append(rotate(images[itr,:,:],90))
    img.append(temp)
    lab.append(labels[itr]-1)
        
    temp=[]
    temp.append(rotate(images[itr,:,:],270))
    img.append(temp)
    lab.append(labels[itr]-1)   
    
    temp = []
    temp.append(cv2.rotate(images[itr,:,:], cv2.ROTATE_180))
    img.append(temp)
    lab.append(labels[itr]-1)
    
img = np.array(img)
lab = np.array(lab)
#img = img.resize((img.shape[0],1,128,128))

lab = lab.astype(int)

class TrainHelper(Dataset):
    def __init__(self):
        self.len = img.shape[0]
        self.img, self.lab = img.astype('float32'), lab.astype('float32')
        self.img = torch.from_numpy(img)
        self.lab = torch.from_numpy(lab)
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        return self.img[idx], self.lab[idx]
traindataset = TrainHelper()
train_loader = DataLoader(dataset = traindataset, batch_size=32,shuffle=True,num_workers=0)

100%|██████████| 3064/3064 [00:13<00:00, 222.84it/s]


In [13]:
train_x, val_x, train_y, val_y = train_test_split(img,lab, test_size = 0.3, shuffle = True)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)
v=[0,0,0]
for i in lab:
    v[i]+=1
print("number of dataapoint in each class are ", v)

number of dataapoint in each class are  [2832, 5704, 3720]


In [14]:
train_y = train_y.astype(int)
val_y = val_y.astype(int)

In [15]:
# These classes helps to load data and implement batch wise training
class TrainHelper(Dataset):
    def __init__(self):
        self.len = train_x.shape[0]
        self.train_x, self.train_y = train_x.astype('float32'), train_y.astype('float32')
        self.train_x = torch.from_numpy(train_x)
        self.train_y = torch.from_numpy(train_y)
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        return self.train_x[idx], self.train_y[idx]
class ValHelper(Dataset):
    def __init__(self):
        self.len = val_x.shape[0]
        self.val_x, self.val_y = val_x.astype('float32'), val_y.astype('float32')
        self.val_x = torch.from_numpy(val_x)
        self.val_y = torch.from_numpy(val_y)
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        return self.val_x[idx], self.val_y[idx]
traindataset = TrainHelper()
valdataset = ValHelper()
train_loader = DataLoader(dataset = traindataset, batch_size=32,shuffle=True,num_workers=0)
val_loader = DataLoader(dataset = valdataset, batch_size=32,shuffle=True,num_workers=0)

In [16]:
class Net50(Module):   
    def __init__(self):
        super(Net50, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 64, kernel_size=10, stride=1, padding=0),
            ReLU(inplace=True),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(64, 128, kernel_size=3, stride=1, padding=2),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout2d(0.10),
            
            Conv2d(128, 256, kernel_size=2, stride=1, padding=2),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),         
            Dropout2d(0.20),
            
            Conv2d(256,12544, kernel_size=7, padding=0),   # This tenique is called fully connected layer using convulation layer.
            ReLU(inplace=True),
            
            Conv2d(12544, 4, kernel_size=1),
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        return x
class Net(Module):
    def __init__(self, model50):
        super(Net, self).__init__()
        self.model50 = model50
        self.cnn_layers2 = Sequential(
            Conv2d(3, 32, kernel_size=2, stride=1, padding=2),
            ReLU(inplace=True),
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(32, 64, kernel_size=3, stride=1, padding=2),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout2d(0.10),   
            
            Conv2d(64, 1024,kernel_size=4),
            ReLU(inplace=True),
            
            Conv2d(1024,3,kernel_size=1)
        )
        
    def forward(self, x):
        x = self.model50(x)
        x = x[:,1:4]
        x = self.cnn_layers2(x)
        return x

In [17]:
model50 = Net50()
#model50.load_state_dict(torch.load('50x50VA98.76epoch94k.pth'))
a,b = next(iter(train_loader))
out50 = model50(a.float())
out50.shape

torch.Size([32, 4, 10, 10])

In [18]:
model = Net(model50)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
#Stochastic Gradient Descent is used
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=10, verbose=True)
# defining the loss function
criterion = CrossEntropyLoss()
l=[0,2,4,8,12,14]
for i in l:
    model.model50.cnn_layers[i].weight.requires_grad = False
    model.model50.cnn_layers[i].bias.requires_grad = False
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
print(model)

Net(
  (model50): Net50(
    (cnn_layers): Sequential(
      (0): Conv2d(1, 64, kernel_size=(10, 10), stride=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout2d(p=0.1, inplace=False)
      (8): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=(2, 2))
      (9): ReLU(inplace=True)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Dropout2d(p=0.2, inplace=False)
      (12): Conv2d(256, 12544, kernel_size=(7, 7), stride=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(12544, 4, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (cnn_layers2): Sequential(

In [19]:
arr=np.random.randn(1,1,128,128)
arr=torch.from_numpy(arr).float()
summary(model,(1,128,128))
model(arr).shape

Layer (type:depth-idx)                   Output Shape              Param #
├─Net50: 1-1                             [-1, 4, 10, 10]           --
|    └─Sequential: 2-1                   [-1, 4, 10, 10]           --
|    |    └─Conv2d: 3-1                  [-1, 64, 119, 119]        (6,464)
|    |    └─ReLU: 3-2                    [-1, 64, 119, 119]        --
|    |    └─BatchNorm2d: 3-3             [-1, 64, 119, 119]        (128)
|    |    └─MaxPool2d: 3-4               [-1, 64, 59, 59]          --
|    |    └─Conv2d: 3-5                  [-1, 128, 61, 61]         (73,856)
|    |    └─ReLU: 3-6                    [-1, 128, 61, 61]         --
|    |    └─MaxPool2d: 3-7               [-1, 128, 30, 30]         --
|    |    └─Dropout2d: 3-8               [-1, 128, 30, 30]         --
|    |    └─Conv2d: 3-9                  [-1, 256, 33, 33]         (131,328)
|    |    └─ReLU: 3-10                   [-1, 256, 33, 33]         --
|    |    └─MaxPool2d: 3-11              [-1, 256, 16, 16]      

torch.Size([1, 3, 1, 1])

In [9]:
ctr1=0
highest_validation_accuracy = 0
def train(epoch,tb):
    global highest_validation_accuracy
    global ctr1
    # ctr1 are used to count the epoch for the test and training mini-batches
    model.train()
    train_loss = []
    correct = 0
    total = 0
    for batch in train_loader:
        data, target = batch
        x_train, y_train = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            x_train = x_train.cuda()
            y_train = y_train.cuda()
        
        output_train = model(x_train.float())
        output_train = output_train.reshape(-1,3)
        #print(output_train.shape,x_train.shape, y_train.shape)
        loss_train = criterion(output_train, y_train)
        train_loss.append(loss_train.item()) 
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        ctr1+=1
        
        _, predicted = torch.max(output_train.data, 1)
        total += y_train.size(0)
        correct += (predicted == y_train).sum().item()    
    tb.add_scalar("Training Loss", loss_train.item(), ctr1)
    tb.add_scalar("Training accuracy", correct/total,ctr1)     
    

    print("Training Loss on loop",ctr1," is ",  np.mean(train_loss))
    print("Training accuracy on loop",ctr1," is ", correct/total)
    validation_loss = []
    correct = 0
    total = 0
    for batch in val_loader:
        data, target = batch
        x_val, y_val = Variable(data), Variable(target)
        if torch.cuda.is_available():
            x_val = x_val.cuda()
            y_val = y_val.cuda()
        output_val = model(x_val.float())
        output_val =  output_val.reshape(-1,3)
        loss_val = criterion(output_val, y_val)
        validation_loss.append(loss_val.item())            
        _, predicted = torch.max(output_val.data, 1)
        total += y_val.size(0)
        correct += (predicted == y_val).sum().item()
    tb.add_scalar("Validation Loss", np.mean(validation_loss), ctr1)
    tb.add_scalar("Validation accuracy", correct/total,ctr1)
    print("Validation Loss on loop",ctr1," is ",  np.mean(validation_loss))
    print("Validation accuracy on loop",ctr1," is ", correct/total)
    if highest_validation_accuracy-0.005 < correct/total:
            highest_validation_accuracy = correct/total
            name = "ValidationAccuracy"+str(highest_validation_accuracy)+"epoch"+str(ctr1)+'.pth'
            torch.save(model.state_dict(),name)
    
    scheduler.step(correct/total)

In [ ]:
n_epochs = 1000
tb = SummaryWriter(comment = 'final model for prediction')
for epoch in range(n_epochs):
    train(epoch,tb)
tb.close()

Training Loss on loop 269  is  0.4051427106376474
Training accuracy on loop 269  is  0.8506818976570696
Validation Loss on loop 269  is  0.19030373890114868
Validation accuracy on loop 269  is  0.9330976339407125
Training Loss on loop 538  is  0.14559456559251233
Training accuracy on loop 538  is  0.95267513696235
Validation Loss on loop 538  is  0.1314434919020404
Validation accuracy on loop 538  is  0.9575741093282567
Training Loss on loop 807  is  0.1119421964795173
Training accuracy on loop 807  is  0.9629327427439095
Validation Loss on loop 807  is  0.11414434736513573
Validation accuracy on loop 807  is  0.96491705194452
Training Loss on loop 1076  is  0.09266814124185356
Training accuracy on loop 1076  is  0.9701596922718265
Validation Loss on loop 1076  is  0.1134188207919183
Validation accuracy on loop 1076  is  0.9635572477563231
Training Loss on loop 1345  is  0.08219173036301058
Training accuracy on loop 1345  is  0.9727240937172165
Validation Loss on loop 1345  is  0.08665